In [2]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Guardar resultados

import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO


# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [3]:
datos = pd.read_csv('datos/230209_ESTACIONES.csv', parse_dates=['FECHAHORA'], index_col=[0])
datos.head()

,ESTACION,FECHAHORA,ANHO,DIA,MES,HORA,MINUTO,MP1,MP2_5,MP10,...,DIA_SEM,AQI_MP2_5,AQI_MP10,MEDICION_DIA,MP1_ANTERIOR,MP2_5_ANTERIOR,MP10_ANTERIOR,TEMPERATURA_PRONOSTICO,HUMEDAD_PRONOSTICO,PRESION_PRONOSTICO
0,1,2019-12-30 13:00:00,2019,30,12,13,0,2.839,3.989,4.599,...,0,72.0,25.0,157,2.609,3.720,4.519,36.5,42.8,995.1
1,1,2019-12-30 13:05:00,2019,30,12,13,5,9.000,13.079,21.319,...,0,72.0,25.0,158,1.919,2.659,10.340,36.7,42.1,994.9
2,1,2019-12-30 13:10:00,2019,30,12,13,10,1.969,2.369,13.170,...,0,72.0,25.0,159,1.769,2.609,3.419,36.6,43.5,994.8
3,1,2019-12-30 13:15:00,2019,30,12,13,15,1.740,2.379,2.429,...,0,72.0,25.0,160,1.480,2.009,2.159,37.0,41.4,994.7
4,1,2019-12-30 13:20:00,2019,30,12,13,20,2.710,4.119,7.710,...,0,72.0,25.0,161,1.250,1.909,2.809,37.1,40.6,994.6


In [4]:
for i in range(1,11):
    
    data = datos[datos['ESTACION'] == i]

    df = data[['FECHAHORA','AQI_MP2_5']]

    df.index = df['FECHAHORA']

    del df['FECHAHORA']

    aqi = df.rolling(1).mean()
    rolling_mean = df.rolling(288*30).mean()
    rolling_std = df.rolling(288*30).std()

    fig = go.Figure()

    fig.add_trace(
        go.Scatter( x = aqi.index, y = aqi.AQI_MP2_5,  name = 'AQI'))

    fig.add_trace(
        go.Scatter( x = aqi.index, y = rolling_mean.AQI_MP2_5,  name = 'Media'))

    fig.add_trace(
        go.Scatter( x = aqi.index, y = rolling_std.AQI_MP2_5, name = 'Desviacion estandar'))

    fig.update_layout( title_text = "Estacion" + str(i) + "AQI, media y desviacion estandar (mensual)")

    fig.write_html('graphs/TSA/30dias_mean_std_estacion'+str(i)+'.html')


In [5]:
from statsmodels.tsa.stattools import adfuller


In [6]:
adfuller_dfs = []
for i in range(1,11):
    data = datos[datos['ESTACION'] == i]

    df = data[['FECHAHORA','AQI_MP2_5']]

    df.index = df['FECHAHORA']

    del df['FECHAHORA']

    adft = adfuller(df,autolag="AIC")

    output_df = pd.DataFrame({"Valores":[i, adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']]  , "Metricas":["Estación","ADF","valor-p","No. de lags utilizado","Numero de observaciones", "valor critico (1%)", "valor crítico (5%)", "valor crítico (10%)"]})
    
    adfuller_dfs.append(output_df)

In [7]:
adfuller_dfs[2]

,Valores,Metricas
0,3.000000e+00,Estación
1,-1.317104e+01,ADF
2,1.250270e-24,valor-p
3,6.200000e+01,No. de lags utilizado
4,1.580500e+05,Numero de observaciones
5,-3.430391e+00,valor critico (1%)
6,-2.861558e+00,valor crítico (5%)
7,-2.566780e+00,valor crítico (10%)


In [8]:
df_arreglada = []
for i in range(0,10):
    
    dfarr = adfuller_dfs[i].T
    dfarr.columns = dfarr.iloc[1]
    dfarr.drop('Metricas', axis = 0, inplace = True)
    df_arreglada.append(dfarr)

df_completo = pd.concat(df_arreglada)

df_completo.reset_index(drop = True)

df_completo

Metricas,Estación,ADF,valor-p,No. de lags utilizado,Numero de observaciones,valor critico (1%),valor crítico (5%),valor crítico (10%)
Valores,1.0,-14.084901,0.0,36.0,157788.0,-3.430391,-2.861558,-2.56678
Valores,2.0,-13.493997,0.0,71.0,158041.0,-3.430391,-2.861558,-2.56678
Valores,3.0,-13.171043,0.0,62.0,158050.0,-3.430391,-2.861558,-2.56678
Valores,4.0,-13.519769,0.0,54.0,158058.0,-3.430391,-2.861558,-2.56678
Valores,5.0,-13.767527,0.0,75.0,158037.0,-3.430391,-2.861558,-2.56678
Valores,6.0,-13.480479,0.0,57.0,158055.0,-3.430391,-2.861558,-2.56678
Valores,7.0,-13.82924,0.0,75.0,158037.0,-3.430391,-2.861558,-2.56678
Valores,8.0,-13.465187,0.0,71.0,158041.0,-3.430391,-2.861558,-2.56678
Valores,9.0,-13.45163,0.0,46.0,158066.0,-3.430391,-2.861558,-2.56678
Valores,10.0,-14.157483,0.0,50.0,158062.0,-3.430391,-2.861558,-2.56678


In [ ]:
df_completo.to_csv('metrics/TSA/ADF_test.csv')

In [9]:
autocorrelaciones = {}

for i in range(1,11):

    data = datos[datos['ESTACION'] == i]

    df = data[['FECHAHORA','AQI_MP2_5']]

    df.index = df['FECHAHORA']

    del df['FECHAHORA']

    print('Estacion '+ str(i))
    print('\n')

    autocorrelation_lag_1h = df['AQI_MP2_5'].autocorr(lag=12)
    print("Hourly Lag: ", autocorrelation_lag_1h)

    autocorrelation_lag_6h = df['AQI_MP2_5'].autocorr(lag=72)
    print("6 hour Lag: ", autocorrelation_lag_6h)

    autocorrelation_lag_12h = df['AQI_MP2_5'].autocorr(lag=144)
    print("12 hour Lag: ", autocorrelation_lag_6h)

    autocorrelation_lag_d = df['AQI_MP2_5'].autocorr(lag=288)
    print("Daily Lag: ", autocorrelation_lag_d)

    autocorrelation_lag_w = df['AQI_MP2_5'].autocorr(lag=288*7)
    print("Weekly Lag: ", autocorrelation_lag_w)

    autocorrelation_lag1 = df['AQI_MP2_5'].autocorr(lag=288*30)
    print("One Month Lag: ", autocorrelation_lag1)

    autocorrelation_lag3 = df['AQI_MP2_5'].autocorr(lag=288*30*3)
    print("Three Month Lag: ", autocorrelation_lag3)

    autocorrelation_lag6 = df['AQI_MP2_5'].autocorr(lag=288*30*6)
    print("Six Month Lag: ", autocorrelation_lag6)

    autocorrelation_lag9 = df['AQI_MP2_5'].autocorr(lag=288*30*9)
    print("Nine Month Lag: ", autocorrelation_lag9)

    print('\n')

    autocorrelaciones[i] = {'ESTACION' : i, '1 hora' : autocorrelation_lag_1h, '6 horas' : autocorrelation_lag_6h, '12 horas' : autocorrelation_lag_12h, 
                            '1 dia' : autocorrelation_lag_d, '1 semana' : autocorrelation_lag_w, '1 mes' : autocorrelation_lag1, '3 meses' : autocorrelation_lag3,
                            '6 meses' : autocorrelation_lag6, '9 meses' : autocorrelation_lag9}

Estacion 1


Hourly Lag:  0.9952971528511562
6 hour Lag:  0.9079315422763662
12 hour Lag:  0.9079315422763662
Daily Lag:  0.41241122933012064
Weekly Lag:  0.16853930687562624
One Month Lag:  0.1624979354973956
Three Month Lag:  0.060722898634752664
Six Month Lag:  -0.050520079791557496
Nine Month Lag:  0.01579881535808525


Estacion 2


Hourly Lag:  0.9934928439175565
6 hour Lag:  0.9152758932334913
12 hour Lag:  0.9152758932334913
Daily Lag:  0.46164656445370816
Weekly Lag:  0.19263891957445328
One Month Lag:  0.1878462922006282
Three Month Lag:  0.07905757955430992
Six Month Lag:  -0.09940023210063625
Nine Month Lag:  -0.06380658907798338


Estacion 3


Hourly Lag:  0.9951221150420569
6 hour Lag:  0.9230152010586729
12 hour Lag:  0.9230152010586729
Daily Lag:  0.49910854065027654
Weekly Lag:  0.18412016481929402
One Month Lag:  0.19795033373017243
Three Month Lag:  0.06667092399381722
Six Month Lag:  -0.09111071769881676
Nine Month Lag:  -0.10978114335884503


Estacion 4


Hourly Lag

In [ ]:
df_prueba = pd.DataFrame.from_dict(autocorrelaciones, orient='index')
df_prueba.to_csv('metrics/TSA/autocorrelaciones.csv')


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import DecomposeResult, seasonal_decompose
from plotly.subplots import make_subplots

def plot_seasonal_decompose(result: DecomposeResult, title="Seasonal Decomposition"):
    return (
        make_subplots(
            rows=4,
            cols=1,
            subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],
        )
        .add_trace(
            go.Scatter(x=result.seasonal.index, y=result.observed, mode="lines"),
            row=1,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.trend.index, y=result.trend, mode="lines"),
            row=2,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.seasonal.index, y=result.seasonal, mode="lines"),
            row=3,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.resid.index, y=result.resid, mode="lines"),
            row=4,
            col=1,
        )
        .update_layout(
            height=900, title=title, margin=dict(t=100), title_x=0.5, showlegend=False
        )
    )


for i in range(1,11):
    data = datos[datos['ESTACION'] == i]

    df = data[['FECHAHORA','AQI_MP2_5']]

    df.index = df['FECHAHORA']

    del df['FECHAHORA']

    for j in [1, 7, 15, 30, 30*3, 30*6, 30*9]:

        decompose = seasonal_decompose(df['AQI_MP2_5'],model='additive', period=288*j)

        print(decompose.resid.shape)


        fig = plot_seasonal_decompose(decompose)

        fig.write_html('graphs/TSA/seasonal_decompose/'+str(j)+'dias_seasonal_decompose_estacion_'+str(i)+'.html')